In [3]:
import numpy as np
import pandas as pd
import json
from elasticsearch import Elasticsearch

## Create index by Elasticsearch ##

### Create indexes for text ###

In [4]:
# Define Elasticsearch instance

es = Elasticsearch(['https://localhost:9201'], basic_auth=('elastic', 'wfRM-cEYuJW0Cxo0nF_z'), verify_certs=False, ssl_show_warn=False)
index_name = 'text_infex'

settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "analysis": {
            "analyzer": {
                "ngram_analyzer": {
                    "type": "custom",
                    "tokenizer": "ngram_tokenizer"
                }
            },
            "tokenizer": {
                "ngram_tokenizer": {
                    "type": "ngram",
                    "min_gram": 5,
                    "max_gram": 5
                }
            },
            "filter": ["lowercase", "stop"]
        }
    },
    "mappings": {
        'properties': {
            'text': {'type': 'text', "analyzer": "ngram_analyzer"}
        }
    }
}


# delete previous index
# response = es.indices.delete(index=index_name)
es.indices.create(index=index_name, body=settings)

/tmp/ipykernel_2858536/1579493722.py:37: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index=index_name, body=settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'text_infex'})

In [ ]:
import jsonlines
folder = './data/missing_value_imputation/'
wiki_pages = []
with jsonlines.open(folder+'wikipages.jsonl') as reader:
    for obj in reader:
        wiki_pages.append(obj)

doc_id = 0
fout = jsonlines.open('doc_ids.jsonl','w')
for i in range(len(wiki_pages)):
    entity_info = wiki_pages[i]
    entity_name = entity_info['entity']
    text = entity_info['text']
    doc = {
        "doc id": doc_id,
        "text": entity_name + ': ' + text
    }
    doc_id += 1
    es.index(index=index_name, document=doc)

fout.close()
print('done')
print(doc_id)

### Create indexes for tuples ###

In [ ]:
# Define Elasticsearch instance
index_name = 'tuple_index'

settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "analysis": {
            "analyzer": {
                "ngram_analyzer": {
                    "type": "custom",
                    "tokenizer": "ngram_tokenizer"
                }
            },
            "tokenizer": {
                "ngram_tokenizer": {
                    "type": "ngram",
                    "min_gram": 5,
                    "max_gram": 5
                }
            },
            "filter": ["lowercase", "stop"]
        }
    },
    "mappings": {
        'properties': {
            'tuple': {'type': 'text', "analyzer": "ngram_analyzer"}
        }
    }
}


# delete previous index
# response = es.indices.delete(index=index_name)
es.indices.create(index=index_name, body=settings)

In [ ]:
global tuple_id
tuple_id = 0

for folder in ['./data/missing_value_imputation/', './data/tabfact/']:
    table_to_page = json.load(open(folder+'table_to_page.json'))
    with open( folder + 'all_csv_ids.json', 'r') as f:
        csv_files = json.load(f)

    for csv_file in csv_files:
        df = pd.read_csv(folder + 'all_csv/' + csv_file, header=0, delimiter='#')
        if csv_file in table_to_page:
            caption = table_to_page[csv_file][0]
        else:
            caption = ""
        headers = df.columns

        for i, row in df.iterrows():
            tuple_string = caption + ' | '
            for j in range(len(row)):
                tuple_string += ' attribute ' + str(headers[j])  + ' value ' + str(row[j])

                doc = {
                "tuple id": tuple_id,
                "tuple": tuple_string
            }

            tuple_id += 1

    print(tuple_id)

### Create indexes for tables ###

In [5]:
# Define Elasticsearch instance
index_name = 'table_index_v3'

settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "analysis": {
            "analyzer": {
                "ngram_analyzer": {
                    "type": "custom",
                    "tokenizer": "ngram_tokenizer"
                }
            },
            "tokenizer": {
                "ngram_tokenizer": {
                    "type": "ngram",
                    "min_gram": 5,
                    "max_gram": 5
                }
            },
            "filter": ["lowercase", "stop"]
        }
    },
    "mappings": {
        'properties': {
            'table': {'type': 'text', "analyzer": "ngram_analyzer"}
        }
    }
}


# delete previous index
response = es.indices.delete(index=index_name)
es.indices.create(index=index_name, body=settings)

/tmp/ipykernel_2858536/3842955694.py:35: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index=index_name, body=settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'table_index_v3'})

In [6]:
global table_id
table_id = 0

for folder in ['./data/missing_value_imputation/', './data/tabfact/']:
    table_to_page = json.load(open(folder+'table_to_page.json'))
    folder += 'all_csv/'
    # 文件夹下面所有的文件
    csv_files = os.listdir(folder)

    for csv_file in csv_files:
        if 'missing_value_imputation' in folder:
            df = pd.read_csv(folder+csv_file, encoding='utf-8')
        else:
            df = pd.read_csv(folder+csv_file, header=0, delimiter='#')
        columns = df.columns.tolist()
        # 读取csv文件中的数据
        data = df.values.tolist()
        # 将list拼接为string
        table_string = ''
        if csv_file in table_to_page:
            table_name = table_to_page[csv_file][0]
        else:
            table_name = ''

        for caption in table_name:
            table_string += caption + '  '
        table_string = table_string[:-2] + '| '
        for attr in columns:
            table_string += attr + ' | '
        for i, row in df.iterrows():
            table_string += 'row | '
            for val in row:
                val = str(val).replace('_', ' ')
                table_string += str(val) + ' | '
            table_string = table_string
      
        doc = {
            "table id": csv_file.split('.')[0],
            "table": table_string,
        }
        table_id += 1
        try:
            es.index(index=index_name, document=doc)
        except:
            print(doc)
            exit(0)
    print(table_id)

2925
19498
